In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard
from keras_tuner import RandomSearch
from sklearn.model_selection import ParameterGrid
train_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    shuffle=False)


Found 7775 images belonging to 8 classes.
Found 1670 images belonging to 8 classes.
Found 1667 images belonging to 8 classes.


In [2]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
tensorboard_callback = TensorBoard(log_dir=r"E:\AInotes\자세교정\모델학습\ResNet\grid_search")

params_grid = {
    'learning_rate': [1e-4, 1e-3],
    'units': [256, 512]
}

# 모델을 빌드하는 함수
def build_and_train_model(learning_rate, units):
    RN50_2 = tf.keras.applications.resnet_v2.ResNet50V2(weights="imagenet",
                                                         include_top=False, 
                                                         input_shape=(128, 128, 3))
    RN50_2.trainable = True

    model = models.Sequential(name="ResNet50v2_RT")
    model.add(RN50_2)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation='relu'))
    model.add(layers.Dense(8, activation='softmax'))
    
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_generator,
                        steps_per_epoch=train_generator.samples // train_generator.batch_size,
                        validation_data=validation_generator, 
                        validation_steps=validation_generator.samples // validation_generator.batch_size,
                        epochs=25,
                        callbacks=[early_stopping, tensorboard_callback])
    return model, history




In [3]:
for params in ParameterGrid(params_grid):
    print("Training with params:", params)
    model, history = build_and_train_model(**params)

Training with params: {'learning_rate': 0.0001, 'units': 256}
Epoch 1/25
485/485 [==============================] - 39s 63ms/step - loss: 2.0425 - accuracy: 0.2187 - val_loss: 1.8609 - val_accuracy: 0.2837
Epoch 2/25
485/485 [==============================] - 24s 49ms/step - loss: 1.6909 - accuracy: 0.3439 - val_loss: 1.6369 - val_accuracy: 0.3696
Epoch 3/25
485/485 [==============================] - 25s 51ms/step - loss: 1.3889 - accuracy: 0.4610 - val_loss: 1.8102 - val_accuracy: 0.3744
Epoch 4/25
485/485 [==============================] - 24s 50ms/step - loss: 1.1219 - accuracy: 0.5680 - val_loss: 1.2986 - val_accuracy: 0.5090
Epoch 5/25
485/485 [==============================] - 23s 48ms/step - loss: 0.8855 - accuracy: 0.6688 - val_loss: 1.2694 - val_accuracy: 0.5349
Epoch 6/25
485/485 [==============================] - 23s 48ms/step - loss: 0.6641 - accuracy: 0.7531 - val_loss: 1.3910 - val_accuracy: 0.5535
Epoch 7/25
485/485 [==============================] - 23s 48ms/step - loss